# Capstone Project Title
___
**Author**: Evan Holder

### Overview <a class="anchor" id="Overview"></a>
___

### Business Problem <a class="anchor" id="Business-Problem"></a>
___

### Data Collection <a class="anchor" id="Data-Collection"></a>
___


### Data Cleaning <a class="anchor" id="Data-Cleaning"></a>
____

### Import Libraries and Functions  <a class="anchor" id="Import-Libraries-and-Funtions"></a>
___
Data manipulation, cleaning, massaging: pandas, numpy<br>
Modeling: sklearn, keras<br>
Plotting: matplotlib<br>
Custom functions: function.py

In [1]:
# Libraries for data cleaning, massaging:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import datetime as dt


# Modeling Libraries
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso

import tensorflow as tf
from tensorflow import keras
from keras import layers, models, regularizers
from tensorflow.keras.layers import TimeDistributed
from sklearn.preprocessing import MinMaxScaler

# Save Models
import pickle

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Manipulate directories 
import os

# Import custom functions
os.chdir('../scripts')
from functions import split_data, sMAPE, SMAPE, compute_metrics, r2,impute_immediate_mean
from functions import resample, plot_metric_range, compile_fit, ensemble_nn
os.chdir('../notebooks')

#from tensorflow.keras.preprocessing import timeseries_dataset_from_array
#from sklearn.model_selection import train_test_split, GridSearchCV
#from sklearn.metrics import r2_score


### Import Data <a class="anchor" id="Import-Data"></a>
___

In [2]:
# Read in data
df_lag = pd.read_csv('../data/clean/df_clean_lag.csv', index_col=0, parse_dates=True)

### Data Preparation <a class="anchor" id="Data-Preparation"></a>
___
In this project, we'll focus on three main algorithms types: Lasso Regression, XGBoost, and Neural Networks.  We'll need to prepare the data in slightly different ways for each of the these model types. Much of the preprocessing was already taken care of as part of the steps list above in [Data Cleaning](#Data-Cleaning). The remaining steps are model specific, and so are prepared below:<br><br>
**Lasso Regression**:<br>
* Encode the categorical features
* Remove mulitcolinearities

**Neural Networks**:<br>
* Encode the categorical features

**XGBoost**: All preprocessing steps were previously 
While not required, we'll scale the continuous features for neural networks. And finally for lasso regression, it would be wise to remove any multicolinearities in the data. In order to run each of these models, I'll copy the dataset and process

### Encode Catergorical Features

In [3]:
# Get Categorical columns
categorical = df_lag.select_dtypes(include='object')

# Instationate wind_dir_coder LabelEncoder, fit
wind_dir_coder = LabelEncoder()
wind_dir_coder.fit(df_lag['wind_madrid_lag'])

# Transform wind_direction cols
for col in categorical.filter(regex='wind').columns:
    df_lag[col] = wind_dir_coder.transform(df_lag[col])
    

# Stack condition columns into single col
stacked_conditions = categorical.filter(regex='condition').stack()

# Instantiate condition_coder LabelEncoder, fit on stacked conditions
condition_coder = LabelEncoder()
condition_coder.fit(stacked_conditions)

# Transform condition cols
for col in categorical.filter(regex='condition').columns:
    df_lag[col] = condition_coder.transform(df_lag[col])

In [4]:
# Get price components not to be used in modeling
price_cols = df_lag.filter(regex='price').columns.to_list()[1:]
price_cols.remove('price_day_ahead')


### Scaling Continuous Features
For neural networks, continuous features do not necessarily need to be scaled. However according to this [article](https://www.sciencedirect.com/science/article/pii/S030626191830196X#s0235), which uses neural networks to predict electrical prices, scaling your continuous features generally increases accuracy of deep learning models on the validation set.  We'll give it a go here and scale the the continuos features between [-1,1] for the neural networks we'll train later.

In [5]:
# Copy the dataframe for neural networks
df_nn = df_lag.drop(columns=price_cols).copy()
continuous = df_nn.select_dtypes(exclude='object').filter(regex='^(?!.*price).*').columns

# Get rid of negatives
time = dt.datetime(2021,3,24,22)
df_nn.loc[time, 'dew_point_bilbao_lag'] = impute_immediate_mean(df_nn['dew_point_bilbao_lag'], time)

# Rescale data [-1,1]
scaler = MinMaxScaler(feature_range=(-1, 1))
df_nn[continuous] = scaler.fit_transform(df_nn[continuous])

### Multicolinearity
One of the assumptions for regression is that featu
res do not contain multicolinearities.  In this section, we'll investigate the predictors and eliminate any multicolinearities in preparation for a lasso regression. We'll need to find out which features are correlated with each other, and remove some of them to rid our dataset of multicolinearities. The steps are outlined below:
1. Copy the dataframe, we'll modify this dataset for use in lasso regression
2. Get the correlations between predictors, sort them in descending order
3. Get the correlations between each individual predictor and the response variable
4. Get the features which have a correlation greater than 0.8, add the feature which correlates less with price_actual to the drop list
5. Drop the features in the drop list

In [6]:
# Copy dataset for lasso regression specific preparation
df_lr = df_lag.drop(columns=price_cols).copy()

# Create correlation matrix predictors to predictors
corr = df_lr.drop(columns='price_actual').corr().abs().stack().reset_index().sort_values(0, ascending=False)
corr.rename(columns={0:'cor'}, inplace=True)  # Rename correlation column
corr = corr.loc[corr['cor']!=1]  # remove correlations between same variables
corr.drop_duplicates(subset='cor', inplace=True) # remove duplicate correlations
corr.reset_index(drop=True, inplace=True) # Reset the index
corr.cor =corr.cor.apply(lambda x: round(x,3))  # Round

# Create correlation matrix predictors to response variable
corr_price = df_lr.corr()['price_actual'].reset_index().sort_values('price_actual', ascending=False)
corr_price = corr_price.loc[corr_price['price_actual']!=1] # remove correlations between same variables
corr_price.reset_index(drop=True, inplace=True)  # Reset the index


drop = []

# For each feature pair where corr > 0.8, add feature with lower corr to price_actual to drop list
for row in range(len(corr.loc[corr.cor>.8])):
    var1 = corr.loc[row,'level_0'] # Get var1 name
    var2 = corr.loc[row,'level_1'] # Get var2 name
    var1_corr = float(corr_price.loc[corr_price['index'] == var1, 'price_actual'])  # Get var1 corr
    var2_corr = float(corr_price.loc[corr_price['index'] == var2, 'price_actual'])  # Get var2 corr
    
    # Add the lower correlation to the drop list
    if var1_corr > var2_corr:
        drop.append(var2)
    else:
        drop.append(var1)
        
# Drop the features in the drop listi
df_lr.drop(columns=drop, inplace=True)

In [7]:
df_xg = df_lag.drop(columns=price_cols)

### Modeling `price_actual` <a class="anchor" id="Modeling-`price_actual`"></a>
___

Create results_actual dataframe to hold results

Add TSO (price_day_ahead) as benchmark prediction to beat

In [8]:
# Benchmark results
TSO_train = df_lag.loc[:'2019', 'price_day_ahead']
TSO_val = df_lag.loc['2020', 'price_day_ahead']

actual_train = df_lag.loc[:'2019', 'price_actual']
actual_val = df_lag.loc['2020', 'price_actual']

# Create dataframe
results_actual = pd.DataFrame(index=['Parameters','SMAPE_train', 'SMAPE_val', 'r2_train', 'r2_val'])

# Add the baseline TSO predictions
results_actual['TSO'] = ['None',
                                    round(sMAPE(actual_train, TSO_train), 3),
                                    round(sMAPE(actual_val, TSO_val), 3),
                                    round(r2(actual_train, TSO_train), 3), 
                                    round(r2(actual_val, TSO_val),3)]
results_actual

,TSO
Parameters,None
SMAPE_train,16.03
SMAPE_val,16.922
r2_train,0.954
r2_val,0.971


### Lasso Regression
To begin we'll split the data into training (2015-2019) and validation (2020). Next, we'll fit a Vanilla lasso regression model with max_iter=10000 to make sure that the model converges.  And finally, we'll compute the output and add it to the results table.

In [9]:
# Split the data
X_train, y_train, X_val, y_val = split_data(df_lr, 2020, 'price_actual')

# Instatiate and fit model on 
lasso = Lasso(max_iter=10000)
lasso.fit(X_train, y_train)

# Add results of vanilla lasso to dataframe
results_actual['Lasso'] = compute_metrics(lasso, 'Vanilla', (X_train, y_train), (X_val, y_val))

# Save model
with open('../models/Lasso.pickle', 'wb') as f:
    pickle.dump(lasso, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973


In [33]:
with open('../models/lasso_vanilla.pickle', 'rb') as file:
    lasso_test = pickle.load(file)

The very first model performed extremely well.  That's great, but it also isn't suprising given that we used the `price_day_ahead` as a predictor which has an r-squared value of 0.971 on the validation set.  The model outperformed the TSO predictions in SMAPE and even increased r-squared by a small margin. Increase r-squared must mean that some of the other features were important in our prediction of `price_actual`.  Below is a plot of the coefficients with the greatest magnitude.

---insert plot here---

It appears that renewable generation and waste generation have a negative relationship with `price_actual` (renewable and waste increase, results in price decrease).  Other than these, it appears that the other features have very little influence on the final price.  

**Recursive Feature Elimination**<br>
As part of the [Lasso](https://github.com/EvanHolder/capstone/blob/main/notebooks/LassoRegression.ipynb) notebook, I ran a recursive feature elimination to see how the model performs with varying amounts of features in the model.  I started with a single feature (`price_day_ahead`), trained a model, and computed its metrics. Then I iteratively added in the next most important feature, trained the new model, and computed its metrics.  This process was repeated until all features were added back into the training set and the metrics were plotted as below.

![RFE_LassoRegression](../images/RFE_LassoRegression.png)

As shown above, when trained on top five features, the model minimizes r-squared.  Below, I'll train the model on these top five features (`price_day_ahead`, `renewable_lag`,`waste_lag`,`oil_lag`,`humidities_seville_lag`).

In [31]:
# Important Features
train_cols = ['humidities_bilbao_lag', 'oil_lag', 'renewable_lag', 'waste_lag', 'price_day_ahead']

# Instatiate and fit model on 
lasso1 = Lasso(max_iter=10000)
lasso1.fit(X_train[train_cols], y_train)

# Add results of vanilla lasso to dataframe
results_actual['Lasso1'] = compute_metrics(lasso1, {'num_features':5}, (X_train[train_cols], y_train), (X_val[train_cols], y_val))

# Save model
with open('../models/Lasso1.pickle', 'wb') as f:
    pickle.dump(lasso1, f)
    
# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},4.134,8.354,0.954,0.971
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn_1-1,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 1}",200.0,200.0,NaN,NaN
nn-24-24,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 162, '...",3.871,3.738,0.981,0.978


In [11]:
# Drop 'price_day_ahead'
X_train1, X_val1 = X_train.drop(columns='price_day_ahead'), X_val.drop(columns='price_day_ahead')

# Instatiate and fit model on 
lasso2 = Lasso(max_iter=10000)
lasso2.fit(X_train1, y_train)

# Add results of vanilla lasso to dataframe
results_actual['Lasso2'] = compute_metrics(lasso2, {'price_day_ahead':False}, (X_train1, y_train), (X_val1, y_val))

# Save Model
with open('../models/Lasso2.pickle', 'wb') as f:
    pickle.dump(lasso2, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557


### XGBoost

In [12]:
# Split the data
X_train, y_train, X_val, y_val = split_data(df_xg, 2020, 'price_actual')

# Instantiate and fit XGBRegressor
xg = XGBRegressor(random_state=17)
xg.fit(X_train, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost'] = compute_metrics(xg, 'Vanilla',(X_train, y_train), (X_val, y_val))

# Save Model
with open('../models/XGBoost.pickle', 'wb') as f:
    pickle.dump(xg, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968


--- write something about trying out different parameter ranges

In [13]:
# Split the data
X_train, y_train, X_val, y_val = split_data(df_xg, 2020, 'price_actual')

# Instantiate and fit XGBRegressor
xg1 = XGBRegressor(random_state=17, max_depth=2)
xg1.fit(X_train, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost1'] = compute_metrics(xg1, {'max_depth':2},(X_train, y_train), (X_val, y_val))

# Save Model
with open('../models/XGBoost1.pickle', 'wb') as f:
    pickle.dump(xg1, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97


Look at feature importances

In [14]:
imp = pd.DataFrame({'importance':xg1.feature_importances_},
                   index=X_train.columns).sort_values(by='importance', ascending=False)
imp.head(10)

,importance
price_day_ahead,0.735356
biomass_lag,0.046353
renewable_lag,0.037915
load_forecast,0.035208
coal_lag,0.034028
waste_lag,0.020222
transmission_fs_lag,0.008342
dew_point_seville_lag,0.007706
solar_lag,0.006477
reservoir_lag,0.006204


Train model without `price_day_ahead`

In [15]:
# Drop price_day_ahead
X_train1, X_val1 = X_train.drop(columns='price_day_ahead'), X_val.drop(columns='price_day_ahead')


# Instantiate and fit XGBRegressor
xg2 = XGBRegressor(random_state=17)
xg2.fit(X_train1, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost2'] = compute_metrics(xg2, {'max_depth':2,
                                                   'price_day_ahead':False},(X_train1, y_train), (X_val1, y_val))
# Save Model
with open('../models/XGBoost2.pickle', 'wb') as f:
    pickle.dump(xg2, f)

# Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427


--- something about tuning parameters without `price_day_ahead`---

In [16]:
# Instantiate and fit XGBRegressor
xg3 = XGBRegressor(random_state=17, max_depth=16)
xg3.fit(X_train1, y_train)

# Compute sMAPE, r2 and add to the table
results_actual['XGBoost3'] = compute_metrics(xg3,
                                             {'max_depth':16, 'price_day_ahead':False},
                                             (X_train1, y_train), 
                                             (X_val1, y_val))
# Save Model
with open('../models/XGBoost3.pickle', 'wb') as f:
    pickle.dump(xg3, f)

#Preview
results_actual.T

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403


### Neural Networks (1-to-1)

In [17]:
# Split Data
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Define input_shape
input_shape = (X_train.shape[1],)

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(layers.Dense(1, activation='relu'))

In [18]:
nn1 = compile_fit(nn, (X_train,y_train), (X_val, y_val), patience=10,
                  loss = tf.keras.metrics.mean_absolute_error)

Epoch 1/200
1369/1369 [==============================] - 2s 1ms/step - loss: 56.7856 - SMAPE: 200.0000 - val_loss: 39.1089 - val_SMAPE: 200.0000
Epoch 2/200
1369/1369 [==============================] - ETA: 0s - loss: 56.7881 - SMAPE: 200.0000 - 2s 1ms/step - loss: 56.7856 - SMAPE: 200.0000 - val_loss: 39.1089 - val_SMAPE: 200.0000
Epoch 3/200
1369/1369 [==============================] - 2s 1ms/step - loss: 56.7857 - SMAPE: 200.0000 - val_loss: 39.1089 - val_SMAPE: 200.0000
Epoch 4/200
1369/1369 [==============================] - 2s 1ms/step - loss: 56.7856 - SMAPE: 200.0000 - val_loss: 39.1089 - val_SMAPE: 200.0000
Epoch 5/200
1369/1369 [==============================] - 2s 1ms/step - loss: 56.7856 - SMAPE: 200.0000 - val_loss: 39.1089 - val_SMAPE: 200.0000
Epoch 6/200
1369/1369 [==============================] - 2s 2ms/step - loss: 56.7857 - SMAPE: 200.0000 - val_loss: 39.1089 - val_SMAPE: 200.0000
Epoch 7/200
1369/1369 [==============================] - 2s 1ms/step - loss: 56.7857 -

In [19]:
params = {'Dense1':59,
          'Dense2':239,
          'Dense3':162,
          'Dense3':1}
results_actual['nn_1-1'] = compute_metrics(nn1, params, (X_train,y_train), (X_val, y_val))

# Save Model
nn1.save('../models/nn1')

# Preview
results_actual.T

C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\holde\anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn_1-1,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 1}",200.0,200.0,NaN,NaN


### Neural Networks (24 to 24)

In [20]:
# Split Data
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

In [21]:
# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24, 24, 24)
X_val, y_val = resample((X_val,y_val), 24, 24, 24)

# Define input_shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))

nn2 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

params = {'Dense1':59,
          'Dense2':239,
          'Dense3':162,
          'TimeDistributed':1}
results_actual['nn-24-24'] = compute_metrics(nn2, params, (X_train,y_train), (X_val, y_val))

# Save model
nn2.save('../models/nn2')

# Preview
results_actual.T

Epoch 1/200
58/58 [==============================] - 1s 7ms/step - loss: 10.4981 - SMAPE: 26.2713 - val_loss: 2.1611 - val_SMAPE: 6.2083
Epoch 2/200
58/58 [==============================] - 0s 4ms/step - loss: 1.9406 - SMAPE: 3.8526 - val_loss: 3.8172 - val_SMAPE: 10.3004
Epoch 3/200
58/58 [==============================] - 0s 5ms/step - loss: 2.0019 - SMAPE: 3.8225 - val_loss: 1.7064 - val_SMAPE: 4.9796
Epoch 4/200
58/58 [==============================] - 0s 5ms/step - loss: 1.7521 - SMAPE: 3.5644 - val_loss: 3.1751 - val_SMAPE: 8.6492
Epoch 5/200
58/58 [==============================] - 0s 5ms/step - loss: 1.7285 - SMAPE: 3.3064 - val_loss: 2.5200 - val_SMAPE: 7.0934
Epoch 6/200
58/58 [==============================] - 0s 5ms/step - loss: 1.6208 - SMAPE: 3.1050 - val_loss: 1.5668 - val_SMAPE: 4.5625
Epoch 7/200
58/58 [==============================] - 0s 5ms/step - loss: 1.8051 - SMAPE: 3.4228 - val_loss: 2.6854 - val_SMAPE: 7.4583
Epoch 8/200
58/58 [==============================] -

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn_1-1,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 1}",200.0,200.0,NaN,NaN
nn-24-24,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 162, '...",3.871,3.738,0.981,0.978


### LSTM

In [22]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Reorganize the training and testing data into batches
X_train, y_train = resample((X_train, y_train), 24*7, 24, 24)
X_val, y_val = resample((X_val,y_val), 24*7, 24, 24)

# Input Shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(60, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

# Compile Fit
nn3 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

# Append results
params = {'LSTM1':60,
          'LSTM2':24,
          'TimeDistributed':1,
          'input_win':'7-days'}

results_actual['LSTM'] = compute_metrics(nn3, params, (X_train,y_train), (X_val,y_val))

# Save Model
nn3.save('../models/nn3')

# Preview
results_actual.T

Epoch 1/200
58/58 [==============================] - 10s 102ms/step - loss: 53.4057 - SMAPE: 175.6884 - val_loss: 33.2597 - val_SMAPE: 144.3217
Epoch 2/200
58/58 [==============================] - 5s 90ms/step - loss: 49.8368 - SMAPE: 153.5010 - val_loss: 31.1744 - val_SMAPE: 128.3129
Epoch 3/200
58/58 [==============================] - 5s 83ms/step - loss: 48.0504 - SMAPE: 143.6363 - val_loss: 29.5401 - val_SMAPE: 116.8996
Epoch 4/200
58/58 [==============================] - 5s 82ms/step - loss: 46.4607 - SMAPE: 135.0932 - val_loss: 27.9949 - val_SMAPE: 106.8921
Epoch 5/200
58/58 [==============================] - 5s 80ms/step - loss: 44.9269 - SMAPE: 127.2520 - val_loss: 26.4899 - val_SMAPE: 97.7824
Epoch 6/200
58/58 [==============================] - 5s 82ms/step - loss: 43.4249 - SMAPE: 120.4168 - val_loss: 25.0227 - val_SMAPE: 89.4552
Epoch 7/200
58/58 [==============================] - 5s 83ms/step - loss: 41.9491 - SMAPE: 113.6031 - val_loss: 23.5879 - val_SMAPE: 81.7939
Epoch 8

58/58 [==============================] - 4s 68ms/step - loss: 4.9382 - SMAPE: 8.9469 - val_loss: 3.8587 - val_SMAPE: 10.7696
Epoch 60/200
58/58 [==============================] - 4s 70ms/step - loss: 4.7818 - SMAPE: 8.8076 - val_loss: 4.1379 - val_SMAPE: 11.5559
Epoch 61/200
58/58 [==============================] - 4s 70ms/step - loss: 4.9300 - SMAPE: 8.9392 - val_loss: 4.4251 - val_SMAPE: 12.0608
Epoch 62/200
58/58 [==============================] - 4s 67ms/step - loss: 4.8954 - SMAPE: 8.9288 - val_loss: 4.0642 - val_SMAPE: 11.2926
Epoch 63/200
58/58 [==============================] - 4s 69ms/step - loss: 4.5529 - SMAPE: 8.3687 - val_loss: 3.9526 - val_SMAPE: 10.9456
Epoch 64/200
58/58 [==============================] - 4s 70ms/step - loss: 4.4620 - SMAPE: 8.2225 - val_loss: 3.9264 - val_SMAPE: 10.9503
Epoch 65/200
58/58 [==============================] - 4s 70ms/step - loss: 4.4057 - SMAPE: 8.0574 - val_loss: 3.9838 - val_SMAPE: 11.0598
Epoch 66/200
58/58 [===========================

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn_1-1,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 1}",200.0,200.0,NaN,NaN
nn-24-24,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 162, '...",3.871,3.738,0.981,0.978


In [23]:
# Input Shape
input_shape = (X_train.shape[1], X_train.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

# Compile Fit
nn4 = compile_fit(nn, (X_train, y_train), (X_val, y_val))

# Append results
params = {'LSTM1':83,
          'LSTM2':24,
          'TimeDistributed':1,
          'input_win':'7-days'}

results_actual['LSTM1'] = compute_metrics(nn4, params, (X_train,y_train), (X_val,y_val))

# Save Model
nn4.save('../models/nn4')

# Preview
results_actual.T

Epoch 1/200
58/58 [==============================] - 8s 103ms/step - loss: 52.9091 - SMAPE: 172.8656 - val_loss: 32.6573 - val_SMAPE: 139.4956
Epoch 2/200
58/58 [==============================] - 6s 98ms/step - loss: 49.3830 - SMAPE: 149.9188 - val_loss: 30.7945 - val_SMAPE: 125.5707
Epoch 3/200
58/58 [==============================] - 6s 96ms/step - loss: 47.6901 - SMAPE: 141.5163 - val_loss: 29.1960 - val_SMAPE: 114.6066
Epoch 4/200
58/58 [==============================] - 6s 96ms/step - loss: 46.1242 - SMAPE: 133.4158 - val_loss: 27.6689 - val_SMAPE: 104.8651
Epoch 5/200
58/58 [==============================] - 6s 96ms/step - loss: 44.6061 - SMAPE: 125.9322 - val_loss: 26.1796 - val_SMAPE: 95.9753
Epoch 6/200
58/58 [==============================] - 5s 94ms/step - loss: 43.1165 - SMAPE: 118.0820 - val_loss: 24.7233 - val_SMAPE: 87.8168
Epoch 7/200
58/58 [==============================] - 6s 96ms/step - loss: 41.6503 - SMAPE: 112.3395 - val_loss: 23.3010 - val_SMAPE: 80.3152
Epoch 8/

58/58 [==============================] - 5s 91ms/step - loss: 4.6707 - SMAPE: 8.3977 - val_loss: 4.1804 - val_SMAPE: 11.6542
Epoch 60/200
58/58 [==============================] - 5s 91ms/step - loss: 4.5772 - SMAPE: 8.5437 - val_loss: 3.9321 - val_SMAPE: 10.9229
Epoch 61/200
58/58 [==============================] - 5s 92ms/step - loss: 4.5810 - SMAPE: 8.5058 - val_loss: 4.0838 - val_SMAPE: 11.2956
Epoch 62/200
58/58 [==============================] - 5s 91ms/step - loss: 4.3850 - SMAPE: 7.9147 - val_loss: 4.4194 - val_SMAPE: 12.2945
Epoch 63/200
58/58 [==============================] - 5s 92ms/step - loss: 4.3502 - SMAPE: 7.8925 - val_loss: 4.2038 - val_SMAPE: 11.7330
Epoch 64/200
58/58 [==============================] - 5s 92ms/step - loss: 4.2466 - SMAPE: 7.7399 - val_loss: 3.9747 - val_SMAPE: 11.0827
Epoch 65/200
58/58 [==============================] - 5s 94ms/step - loss: 4.1603 - SMAPE: 7.5915 - val_loss: 3.9382 - val_SMAPE: 10.9863
Epoch 66/200
58/58 [===========================

,Parameters,SMAPE_train,SMAPE_val,r2_train,r2_val
TSO,None,16.03,16.922,0.954,0.971
Lasso,Vanilla,3.021,5.869,0.977,0.973
Lasso1,{'num_features': 5},3.367,5.056,0.971,0.969
Lasso2,{'price_day_ahead': False},11.811,32.664,0.676,0.557
XGBoost,Vanilla,1.248,6.668,0.996,0.968
XGBoost1,{'max_depth': 2},2.465,5.85,0.984,0.97
XGBoost2,"{'max_depth': 2, 'price_day_ahead': False}",4.331,27.241,0.953,0.427
XGBoost3,"{'max_depth': 16, 'price_day_ahead': False}",0.026,24.84,1.0,0.403
nn_1-1,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 1}",200.0,200.0,NaN,NaN
nn-24-24,"{'Dense1': 59, 'Dense2': 239, 'Dense3': 162, '...",3.871,3.738,0.981,0.978


### LSTM-DNN

In [6]:
# Split the data into train and validation
X_train, y_train, X_val, y_val = split_data(df_nn, 2020, 'price_actual')

# Get the x cols for lstm network, lagged cols
X_train_lstm = X_train.filter(regex='lag')
X_train_dnn = X_train.drop(columns=X_train_lstm.columns)

# Get the x cols for dnn network, forecast cols
X_val_lstm = X_val.filter(regex='lag')
X_val_dnn = X_val.drop(columns=X_val_lstm.columns)

# Reorganize the training and testing data into batches
X_train_dnn, y_train_dnn = resample((X_train_dnn, y_train), 24, 24, 24)
X_val_dnn, y_val_dnn = resample((X_val_dnn, y_val), 24, 24, 24)

# LSTM
X_train_lstm, y_train_lstm = resample((X_train_lstm, y_train), 24, 24, 24)
X_val_lstm, y_val_lstm = resample((X_val_lstm, y_val), 24, 24, 24)

# Input Shape
input_shape = (X_train_dnn.shape[1], X_train_dnn.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.Dense(59, activation='relu', input_shape=input_shape))
nn.add(layers.Dense(239, activation='relu'))
nn.add(layers.Dense(162, activation='relu'))
nn.add(TimeDistributed(layers.Dense(1)))

dnn = compile_fit(nn, (X_train_dnn, y_train_dnn), (X_val_dnn, y_val_dnn))

Epoch 1/200
58/58 [==============================] - 1s 6ms/step - loss: 12.3746 - SMAPE: 32.1995 - val_loss: 1.9944 - val_SMAPE: 6.8858
Epoch 2/200
58/58 [==============================] - 0s 4ms/step - loss: 2.7985 - SMAPE: 6.5479 - val_loss: 2.1330 - val_SMAPE: 6.3469
Epoch 3/200
58/58 [==============================] - 0s 4ms/step - loss: 2.6315 - SMAPE: 5.5008 - val_loss: 1.9710 - val_SMAPE: 5.9442
Epoch 4/200
58/58 [==============================] - 0s 3ms/step - loss: 2.5122 - SMAPE: 4.9637 - val_loss: 2.3746 - val_SMAPE: 6.9121
Epoch 5/200
58/58 [==============================] - 0s 4ms/step - loss: 2.4242 - SMAPE: 4.7868 - val_loss: 2.0387 - val_SMAPE: 6.1979
Epoch 6/200
58/58 [==============================] - 0s 4ms/step - loss: 2.5448 - SMAPE: 4.9840 - val_loss: 1.8914 - val_SMAPE: 5.7971
Epoch 7/200
58/58 [==============================] - 0s 4ms/step - loss: 2.4210 - SMAPE: 4.6925 - val_loss: 2.0036 - val_SMAPE: 6.1003
Epoch 8/200
58/58 [==============================] - 

In [7]:
compute_metrics(dnn, 'None',(X_train_dnn, y_train_dnn), (X_val_dnn, y_val_dnn))

['None', 5.79, 5.482, 0.96, 0.963]

In [8]:
# Input Shape
input_shape = (X_train_lstm.shape[1], X_train_lstm.shape[2])

# Instantiate model and build layers
nn = models.Sequential()
nn.add(layers.LSTM(83, activation='tanh', input_shape=input_shape))
nn.add(layers.RepeatVector(y_train_lstm.shape[1]))
nn.add(layers.LSTM(24, activation='tanh', return_sequences=True))
nn.add(TimeDistributed(layers.Dense(1)))

lstm = compile_fit(nn, (X_train_lstm, y_train_lstm), (X_val_lstm, y_val_lstm))

Epoch 1/200
58/58 [==============================] - 3s 25ms/step - loss: 52.6457 - SMAPE: 170.9025 - val_loss: 32.5681 - val_SMAPE: 138.8117
Epoch 2/200
58/58 [==============================] - 1s 18ms/step - loss: 49.3168 - SMAPE: 150.6581 - val_loss: 30.6863 - val_SMAPE: 124.8068
Epoch 3/200
58/58 [==============================] - 1s 18ms/step - loss: 47.5523 - SMAPE: 141.1469 - val_loss: 29.0365 - val_SMAPE: 113.5617
Epoch 4/200
58/58 [==============================] - 1s 18ms/step - loss: 45.9508 - SMAPE: 132.8189 - val_loss: 27.4837 - val_SMAPE: 103.7307
Epoch 5/200
58/58 [==============================] - 1s 18ms/step - loss: 44.4116 - SMAPE: 124.6981 - val_loss: 25.9782 - val_SMAPE: 94.8193
Epoch 6/200
58/58 [==============================] - 1s 18ms/step - loss: 42.9081 - SMAPE: 118.0794 - val_loss: 24.5145 - val_SMAPE: 86.6892
Epoch 7/200
58/58 [==============================] - 1s 19ms/step - loss: 41.4330 - SMAPE: 110.9771 - val_loss: 23.0847 - val_SMAPE: 79.2133
Epoch 8/2

In [18]:
#LSTM_DNN = ensemble_nn([dnn, lstm])

nn5 = compile_fit(LSTM_DNN, ([X_train_dnn, X_train_lstm], y_train_dnn), ([X_val_dnn, X_val_lstm], y_val_dnn))

Epoch 1/200
58/58 [==============================] - 3s 22ms/step - loss: 2.0247 - SMAPE: 3.9934 - val_loss: 3.3477 - val_SMAPE: 9.3828
Epoch 2/200
58/58 [==============================] - 1s 11ms/step - loss: 2.0468 - SMAPE: 3.9272 - val_loss: 2.9500 - val_SMAPE: 8.4063
Epoch 3/200
58/58 [==============================] - 1s 12ms/step - loss: 2.0287 - SMAPE: 3.8945 - val_loss: 3.2214 - val_SMAPE: 9.0830
Epoch 4/200
58/58 [==============================] - 1s 10ms/step - loss: 2.0226 - SMAPE: 3.9312 - val_loss: 2.9404 - val_SMAPE: 8.3859
Epoch 5/200
58/58 [==============================] - 1s 10ms/step - loss: 2.0256 - SMAPE: 4.0693 - val_loss: 3.0385 - val_SMAPE: 8.6289
Epoch 6/200
58/58 [==============================] - 1s 11ms/step - loss: 2.0177 - SMAPE: 3.9020 - val_loss: 3.1071 - val_SMAPE: 8.7770
Epoch 7/200
58/58 [==============================] - 1s 10ms/step - loss: 2.0150 - SMAPE: 3.8586 - val_loss: 3.4189 - val_SMAPE: 9.5543
Epoch 8/200
58/58 [=============================

In [20]:
nn5.save('../models/nn5')

INFO:tensorflow:Assets written to: ../models/nn5\assets


INFO:tensorflow:Assets written to: ../models/nn5\assets


### XGBoost

### Neural Networks

## Results
___

## Conclusion
___

## Next Steps
___